In [1]:
pip install -q -U "tensorflow-text==2.11.*"

In [2]:
pip install -q tf-models-official==2.11.0

In [3]:
import os
import shutil

import tensorflow as tf
import tensorflow_hub as hub
import tensorflow_text as text
from official.nlp import optimization  # to create AdamW optimizer

import matplotlib.pyplot as plt

tf.get_logger().setLevel('ERROR')

In [4]:
import pandas as pd


In [5]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [6]:
path = "/content/drive/MyDrive/Data/clean_output.csv"

In [7]:
data_df = pd.read_csv(path)

In [8]:
data_df.head()

,Unnamed: 0,cleaned_title,cleaned_text,target
0,0,smell hillarys fear,daniel greenfield shillman journalism fellow f...,1
1,1,watch exact moment paul ryan committed politic...,google pinterest digg linkedin reddit stumbleu...,1
2,2,kerry go paris gesture sympathy,yous secretary state john f kerry said monday ...,0
3,3,bernie supporters twitter erupt anger dnc trie...,kaydee king kaydeeking november 9 2016 lesson ...,1
4,4,battle new york primary matters,primary day new york frontrunners hillary clin...,0


In [9]:
model_df1 = data_df[['cleaned_title','cleaned_text','target']]

In [10]:
model_df1.head()

,cleaned_title,cleaned_text,target
0,smell hillarys fear,daniel greenfield shillman journalism fellow f...,1
1,watch exact moment paul ryan committed politic...,google pinterest digg linkedin reddit stumbleu...,1
2,kerry go paris gesture sympathy,yous secretary state john f kerry said monday ...,0
3,bernie supporters twitter erupt anger dnc trie...,kaydee king kaydeeking november 9 2016 lesson ...,1
4,battle new york primary matters,primary day new york frontrunners hillary clin...,0


In [11]:
y= model_df1['target'].values
X= model_df1['cleaned_text'].values



In [12]:
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(X, y, random_state=21)

In [13]:
text_input = tf.keras.layers.Input(shape=(), dtype=tf.string)
dense_layer = tf.keras.layers.Dense(1, activation='sigmoid', name='classifier')
preprocessor = hub.KerasLayer(
    "https://tfhub.dev/tensorflow/bert_en_uncased_preprocess/3")
encoder_inputs = preprocessor(text_input)



In [14]:
encoder = hub.KerasLayer(
    "https://tfhub.dev/tensorflow/small_bert/bert_en_uncased_L-4_H-256_A-4/2",
    trainable=True)
outputs = encoder(encoder_inputs)
pooled_output = outputs["pooled_output"]      # [batch_size, 256].
sequence_output = outputs["sequence_output"]  # [batch_size, seq_length, 256].
classification_out = dense_layer(pooled_output)
batch_size = 256

#50/50 [==============================] - 44s 861ms/step - loss: 0.4259 - binary_accuracy: 0.7961
#Loss: 0.4258823096752167
#Accuracy: 0.7960858345031738

In [ ]:
#encoder = hub.KerasLayer(
#    "https://tfhub.dev/tensorflow/small_bert/bert_en_uncased_L-2_H-128_A-2/2",
#    trainable=True)
#outputs = encoder(encoder_inputs)
#pooled_output = outputs["pooled_output"]      # [batch_size, 128].
#classification_out = dense_layer(pooled_output)
#sequence_output = outputs["sequence_output"]  # [batch_size, seq_length, 128].
#batch_size = 128

#50/50 [==============================] - 10s 173ms/step - loss: 0.5648 - binary_accuracy: 0.7058
#Loss: 0.5648084282875061
#Accuracy: 0.7058081030845642

In [15]:
text_classification_model = tf.keras.Model(text_input, classification_out)

In [16]:
loss = tf.keras.losses.BinaryCrossentropy(from_logits=False)
metrics = tf.metrics.BinaryAccuracy()

In [17]:
import numpy as np

In [18]:
epochs = 5

import math
steps_per_epoch = math.ceil(len(X_train) / batch_size)
num_train_steps = steps_per_epoch * epochs
num_warmup_steps = int(0.1*num_train_steps)

init_lr = 3e-5
optimizer = optimization.create_optimizer(init_lr=init_lr,
                                          num_train_steps=num_train_steps,
                                          num_warmup_steps=num_warmup_steps,
                                          optimizer_type='adamw')

In [19]:
text_classification_model.compile(
    optimizer = optimizer,
    loss = loss,
    metrics = metrics
)

In [20]:
X_train = np.asarray(X_train).astype(str)
X_test = np.asarray(X_test).astype(str)
y_train = np.asarray(y_train).astype(np.int)
y_test = np.asarray(y_test).astype(np.int)

<ipython-input-20-00491bdb6ae5>:3: DeprecationWarning: `np.int` is a deprecated alias for the builtin `int`. To silence this warning, use `int` by itself. Doing this will not modify any behavior and is safe. When replacing `np.int`, you may wish to use e.g. `np.int64` or `np.int32` to specify the precision. If you wish to review your current use, check the release note link for additional information.
Deprecated in NumPy 1.20; for more details and guidance: https://numpy.org/devdocs/release/1.20.0-notes.html#deprecations
  y_train = np.asarray(y_train).astype(np.int)
<ipython-input-20-00491bdb6ae5>:4: DeprecationWarning: `np.int` is a deprecated alias for the builtin `int`. To silence this warning, use `int` by itself. Doing this will not modify any behavior and is safe. When replacing `np.int`, you may wish to use e.g. `np.int64` or `np.int32` to specify the precision. If you wish to review your current use, check the release note link for additional information.
Deprecated in NumPy 1

In [21]:
model = text_classification_model.fit(X_train, y_train, epochs=epochs)

Epoch 1/5
149/149 [==============================] - 420s 3s/step - loss: 0.4974 - binary_accuracy: 0.7582
Epoch 2/5
149/149 [==============================] - 413s 3s/step - loss: 0.4314 - binary_accuracy: 0.8104
Epoch 3/5
149/149 [==============================] - 405s 3s/step - loss: 0.4307 - binary_accuracy: 0.8043
Epoch 4/5
149/149 [==============================] - 410s 3s/step - loss: 0.4312 - binary_accuracy: 0.8087
Epoch 5/5
149/149 [==============================] - 405s 3s/step - loss: 0.4306 - binary_accuracy: 0.8061


In [22]:
loss, accuracy = text_classification_model.evaluate(X_test, y_test)

print(f'Loss: {loss}')
print(f'Accuracy: {accuracy}')

50/50 [==============================] - 44s 861ms/step - loss: 0.4259 - binary_accuracy: 0.7961
Loss: 0.4258823096752167
Accuracy: 0.7960858345031738
